In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import sys

# --- 1. Cấu hình đường dẫn để import từ 'src' ---

# Lấy đường dẫn đến notebook hiện tại (nó sẽ là .../[PROJECT ROOT]/notebooks)
notebook_dir = Path.cwd()

# Đi lùi 1 cấp để lấy thư mục gốc của dự án
project_root = notebook_dir.parent

# Tạo đường dẫn đến thư mục 'src'
src_dir = project_root / 'src'

# Thêm 'src' vào sys.path để Python biết chỗ tìm file .py
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

print(f"Đã thêm vào sys.path: {src_dir}")

# --- 2. Import hàm từ file src/data_cleaning.py ---
try:
    from data_cleaning import preprocess_and_filter_data
    print("Import hàm 'preprocess_and_filter_data' từ 'src' thành công!")
except ImportError as e:
    print(f"LỖI: Không thể import hàm. Kiểm tra lại file 'src/data_cleaning.py'. Lỗi: {e}")


# --- 3. Cấu hình đường dẫn Data ---
# Sử dụng 'project_root' đã định nghĩa ở trên
DATA_DIR = project_root / 'data'
CONFIG_FILE_PATH = DATA_DIR / 'to_hop.json'
OUTPUT_FILE_PATH = DATA_DIR / 'diem_thi_da_xu_ly.csv'
FILE_PATTERN = 'diem_thi_*.csv'

print(f"Thư mục Data: {DATA_DIR}")
print(f"File Config: {CONFIG_FILE_PATH}")

Đã thêm vào sys.path: d:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\src
Import hàm 'preprocess_and_filter_data' từ 'src' thành công!
Thư mục Data: d:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\data
File Config: d:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\data\to_hop.json


In [2]:
# === KHỐI THỰC THI "MAIN" ===

# 1. Tìm tất cả các file data thô
print(f"Đang tìm các tệp theo mẫu: {DATA_DIR / FILE_PATTERN}")
all_files = list(DATA_DIR.glob(FILE_PATTERN))

if not all_files:
    print(f"Lỗi: Không tìm thấy tệp CSV nào khớp với mẫu '{FILE_PATTERN}' trong thư mục '{DATA_DIR}'.")
else:
    print(f"Đã tìm thấy {len(all_files)} tệp. Đang tiến hành gộp...")
    
    # 2. Đọc và gộp các file
    df_list = []
    for f in all_files:
        print(f" - Đang đọc file: {f.name}")
        try:
            df_temp = pd.read_csv(f)
            df_list.append(df_temp)
        except Exception as e:
            print(f"   Lỗi khi đọc file {f.name}: {e}")

    # Gộp tất cả lại thành 1 DataFrame
    if not df_list:
        print("Không có dữ liệu nào để xử lý.")
    else:
        df_raw = pd.concat(df_list, ignore_index=True)
        print(f"Tổng số thí sinh đã gộp (chưa lọc): {len(df_raw)}")
        print("-" * 30)

        # 3. Gọi hàm làm sạch (đã import từ src)
        df_final = preprocess_and_filter_data(df_raw, CONFIG_FILE_PATH)

        # 4. Lưu kết quả
        if df_final is not None and not df_final.empty:
            print("-" * 30)
            print(f"\n--- Dữ liệu cuối cùng ({len(df_final)} thí sinh) ---")
            
            # Hiển thị 5 dòng đầu tiên trong notebook
            display(df_final.head()) 

            # 5. Lưu tệp
            df_final.to_csv(OUTPUT_FILE_PATH, index=False, encoding='utf-8-sig')
            print(f"\nĐã lưu dữ liệu đã xử lý vào tệp: {OUTPUT_FILE_PATH}")
        else:
            print("\nKhông có dữ liệu nào còn lại sau khi lọc.")

Đang tìm các tệp theo mẫu: d:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\data\diem_thi_*.csv
Đã tìm thấy 2 tệp. Đang tiến hành gộp...
 - Đang đọc file: diem_thi_toanquoc.csv
 - Đang đọc file: diem_thi_toan_quoc_1.csv
Tổng số thí sinh đã gộp (chưa lọc): 889388
------------------------------
Bắt đầu xử lý. Số lượng ban đầu: 889388
Sau khi loại bỏ SBD rỗng: 889388
Sau khi loại bỏ SBD trùng lặp: 889388
Sau khi loại bỏ điểm liệt (<= 1.0): 888560
Đã tải thành công 49 tổ hợp từ 'd:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\data\to_hop.json'
Đang tính toán điểm các tổ hợp...
Sau khi lọc theo tổ hợp >= 15đ: 757680
Hoàn tất xử lý.
------------------------------

--- Dữ liệu cuối cùng (757680 thí sinh) ---


,MA_TINH,SBD,Toán,Văn,Ngoại ngữ,Lí,Hóa,Sinh,Sử,Địa,...,D08,D09,D10,D11,D12,D13,D14,D15,D66,D84
0,32,32000006,2.85,7.75,<NA>,4.0,<NA>,<NA>,5.35,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,32,32000017,5.35,8.25,<NA>,8.5,8.25,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,32,32000015,3.85,6.25,<NA>,6.0,3.25,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,32,32000016,2.75,7.0,<NA>,<NA>,<NA>,<NA>,5.85,4.95,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,32,32000001,4.35,6.5,<NA>,<NA>,4.5,5.0,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>



Đã lưu dữ liệu đã xử lý vào tệp: d:\Documents University\Semester 5\Intro DS\Project\introduction-to-data-science-feature-crawl-diem-thi\data\diem_thi_da_xu_ly.csv
